In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
#from groq import Groq
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

c:\Users\sreer\miniconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone(api_key = os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '35163baf9e3f9dc91c1f762437103637', 'Date': 'Sat, 31 Aug 2024 16:48:28 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [3]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Introduction to Computer Science',
  'stars': 4,
  'review': 'Dr. Smith explains concepts clearly but can be a bit fast-paced.'},
 {'professor': 'Dr. Jane Doe',
  'subject': 'Calculus I',
  'stars': 5,
  'review': 'Excellent professor! She makes complex topics easy to understand.'},
 {'professor': 'Dr. Emily Brown',
  'subject': 'Physics II',
  'stars': 3,
  'review': 'Content is tough, and lectures are a bit dry, but she’s knowledgeable.'},
 {'professor': 'Dr. Michael Johnson',
  'subject': 'World History',
  'stars': 2,
  'review': 'Lectures are disorganized and hard to follow.'},
 {'professor': 'Dr. Alice White',
  'subject': 'Organic Chemistry',
  'stars': 5,
  'review': 'Very thorough and engaging. Her exams are challenging but fair.'},
 {'professor': 'Dr. Robert Green',
  'subject': 'Microeconomics',
  'stars': 4,
  'review': 'Great lecturer, but office hours are often crowded.'},
 {'professor': 'Dr. Susan Lee',
  'subject': 'Introdu

In [4]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model = "text-embedding-3-small",
    )

    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [5]:
#available_models = client.models.list()
#print(available_models)
processed_data[0]

{'values': [-0.017332475632429123,
  0.040877409279346466,
  -0.0336153544485569,
  -0.021686868742108345,
  0.0038898782804608345,
  0.012190885841846466,
  0.0076733785681426525,
  0.01687859743833542,
  0.037359852343797684,
  -0.02195635996758938,
  0.006414575967937708,
  -0.015176553279161453,
  -0.010829251259565353,
  -0.012864611111581326,
  0.05440865084528923,
  -0.0577276386320591,
  -0.04819619283080101,
  -0.04300496354699135,
  -0.002891700714826584,
  0.015474410727620125,
  0.01248165126889944,
  -0.005627380684018135,
  0.014637572690844536,
  -0.0028846089262515306,
  -0.05596885830163956,
  0.0075386338867247105,
  0.050040073692798615,
  0.034182704985141754,
  0.03094882145524025,
  -0.01074414886534214,
  0.06581234186887741,
  -0.022750645875930786,
  0.038863323628902435,
  -0.01659492217004299,
  -0.04439496248960495,
  0.03775699436664581,
  -0.00690745934844017,
  0.016467269510030746,
  0.004049444571137428,
  0.0019449391402304173,
  0.004630976356565952,


In [6]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [7]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}